# **Extract the dataset.**

In [4]:
import zipfile
import os

# Assuming the uploaded file is named 'dataset.zip'
uploaded_zip_path = '/content/dataset.zip'
extract_path = '/content/NewDataSet'  # Path to extract the dataset

# Create the extract directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(uploaded_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f'Dataset extracted to {extract_path}')


Dataset extracted to /content/NewDataSet


In [5]:
# List files in the extracted directory
extracted_files = os.listdir(extract_path)
print(f'Extracted files: {extracted_files}')


Extracted files: ['__MACOSX', 'dataset']


In [6]:
import shutil
from sklearn.model_selection import train_test_split

# Define paths
test_dir = '/content/NewDataSet/dataset/test'
train_dir = '/content/NewDataSet/dataset/train'
temp_train_dir = '/content/NewDataSet/dataset/temp_train'
temp_validation_dir = '/content/NewDataSet/dataset/temp_validation'

# Create temporary directories for split data
os.makedirs(temp_train_dir, exist_ok=True)
os.makedirs(temp_validation_dir, exist_ok=True)

# Function to split data into training and validation sets
def split_data(source_dir, train_dir, val_dir, train_size=0.7):
    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if os.path.isdir(class_path):
            images = os.listdir(class_path)
            train_images, val_images = train_test_split(images, train_size=train_size)

            os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

            for image in train_images:
                shutil.copy(os.path.join(class_path, image), os.path.join(train_dir, class_name, image))
            for image in val_images:
                shutil.copy(os.path.join(class_path, image), os.path.join(val_dir, class_name, image))

# Split the test data into training and validation sets
split_data(test_dir, temp_train_dir, temp_validation_dir, train_size=0.7)


# **Q2**

In [8]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [9]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from wandb.integration.keras import WandbCallback

In [10]:
!pip install --upgrade wandb

In [11]:
import os
import wandb

# Set WANDB_API_KEY environment variable (replace 'your_api_key' with your actual API key)
os.environ['WANDB_API_KEY'] = '5237a13f8f1acba19960eff228299c80defa0a3d'

# Initialize wandb
wandb.login()
wandb.init(project="Q2", entity="praba00021")


wandb: Currently logged in as: praba00021. Use `wandb login --relogin` to force relogin


In [17]:
# Define paths
train_data_dir = '/content/NewDataSet/dataset/train'
test_data_dir = '/content/NewDataSet/dataset/test'
val_data_dir = '/content/NewDataSet/dataset/test'

In [18]:
# Set aside 10% of the training data for hyperparameter tuning
train_images = []
class_labels = []
for class_name in os.listdir(train_data_dir):
    class_path = os.path.join(train_data_dir, class_name)
    if os.path.isdir(class_path):  # Ensure it's a directory
        for img in os.listdir(class_path):
            if img != '.DS_Store':  # Ignore .DS_Store files
                train_images.append(os.path.join(class_path, img))
                class_labels.append(class_name)

# Split data into training and validation sets
hyperparam_tuning_data, val_data, hyperparam_tuning_labels, val_labels = train_test_split(
    train_images, class_labels, test_size=0.1, random_state=42, stratify=class_labels)


In [19]:
# Move validation data to a separate directory
os.makedirs(val_data_dir, exist_ok=True)

for img_path, label in zip(val_data, val_labels):
    class_dir = os.path.join(val_data_dir, label)
    os.makedirs(class_dir, exist_ok=True)
    shutil.move(img_path, class_dir)

# Prepare data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


Found 585 images belonging to 2 classes.
Found 473 images belonging to 2 classes.


In [20]:
# Calculate the number of classes
num_classes = len([d for d in os.listdir(train_data_dir) if os.path.isdir(os.path.join(train_data_dir, d))])


In [21]:
# Define function to build the model
def build_model(input_shape, num_classes, filters_per_layer, dropout_rate, use_batch_normalization):
    model = Sequential()

    for i, filters in enumerate(filters_per_layer):
        if i == 0:
            model.add(Conv2D(filters, (3, 3), padding='same', input_shape=input_shape))
        else:
            model.add(Conv2D(filters, (3, 3), padding='same'))

        if use_batch_normalization:
            model.add(BatchNormalization())

        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        if dropout_rate:
            model.add(Dropout(dropout_rate))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model


In [22]:
# Define hyperparameters to tune
filters_per_layer_options = [
    [32, 32, 32, 32, 32],  # Same number of filters in all layers
    [32, 64, 128, 256, 512],  # Doubling in each subsequent layer
    [512, 256, 128, 64, 32]  # Halving in each subsequent layer
]
dropout_rates = [0.2, 0.3]
use_batch_normalization = [True, False]


In [23]:
# Ensure filters_per_layer_options, dropout_rates, use_batch_normalization, train_generator, val_generator are defined correctly

from tensorflow.keras.layers import BatchNormalization, Dropout
from wandb.integration.keras import WandbCallback

# Train and evaluate models with different hyperparameter configurations
for filters_per_layer in filters_per_layer_options:
    for dropout_rate in dropout_rates:
        for batch_norm in use_batch_normalization:
            # Build model
            model = build_model(input_shape=(64, 64, 3), num_classes=num_classes, filters_per_layer=filters_per_layer, dropout_rate=dropout_rate, use_batch_normalization=batch_norm)

            # Compile model
            model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

            # Train model
            model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=[WandbCallback()])


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
19/19 [==============================] - ETA: 0s - loss: 0.8218 - accuracy: 0.5316

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240528_070410-7hqkl37e/files/model-best)... Done. 0.0s


19/19 [==============================] - 18s 757ms/step - loss: 0.8218 - accuracy: 0.5316 - val_loss: 0.6959 - val_accuracy: 0.4989
Epoch 2/10
19/19 [==============================] - ETA: 0s - loss: 0.7657 - accuracy: 0.5316

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240528_070410-7hqkl37e/files/model-best)... Done. 0.0s


...

19/19 [==============================] - 8s 417ms/step - loss: 0.6918 - accuracy: 0.5043 - val_loss: 0.6901 - val_accuracy: 0.6321
Epoch 9/10
19/19 [==============================] - 7s 375ms/step - loss: 0.6940 - accuracy: 0.5214 - val_loss: 0.6945 - val_accuracy: 0.4989
Epoch 10/10
19/19 [==============================] - 5s 267ms/step - loss: 0.6948 - accuracy: 0.5009 - val_loss: 0.6938 - val_accuracy: 0.4989
Epoch 1/10
19/19 [==============================] - 20s 808ms/step - loss: 3.2618 - accuracy: 0.5060 - val_loss: 0.8940 - val_accuracy: 0.5011
Epoch 2/10
19/19 [==============================] - 15s 783ms/step - loss: 0.7092 - accuracy: 0.5179 - val_loss: 0.6959 - val_accuracy: 0.5095
Epoch 3/10
19/19 [==============================] - 15s 780ms/step - loss: 0.7039 - accuracy: 0.5128 - val_loss: 0.6941 - val_accuracy: 0.4989
Epoch 4/10
19/19 [==============================] - 15s 782ms/step - loss: 0.7084 - accuracy: 0.5333 - val_loss: 0.7000 - val_accuracy: 0.4989
Epoch 5/10
1

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240528_070410-7hqkl37e/files/model-best)... Done. 0.2s


19/19 [==============================] - 15s 809ms/step - loss: 0.6917 - accuracy: 0.5179 - val_loss: 0.6857 - val_accuracy: 0.5835
Epoch 8/10
19/19 [==============================] - ETA: 0s - loss: 0.6823 - accuracy: 0.5880

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240528_070410-7hqkl37e/files/model-best)... Done. 0.2s


19/19 [==============================] - 16s 815ms/step - loss: 0.6823 - accuracy: 0.5880 - val_loss: 0.6726 - val_accuracy: 0.5433
Epoch 9/10
19/19 [==============================] - 12s 623ms/step - loss: 0.6845 - accuracy: 0.5504 - val_loss: 0.6845 - val_accuracy: 0.5518
Epoch 10/10
19/19 [==============================] - 13s 675ms/step - loss: 0.6890 - accuracy: 0.5231 - val_loss: 0.6753 - val_accuracy: 0.5560
Epoch 1/10
19/19 [==============================] - 20s 839ms/step - loss: 2.5580 - accuracy: 0.5043 - val_loss: 0.6961 - val_accuracy: 0.4989
Epoch 2/10
19/19 [==============================] - 15s 778ms/step - loss: 0.6783 - accuracy: 0.5846 - val_loss: 0.7051 - val_accuracy: 0.4989
Epoch 3/10
19/19 [==============================] - 15s 804ms/step - loss: 0.6673 - accuracy: 0.6308 - val_loss: 0.7393 - val_accuracy: 0.4989
Epoch 4/10
19/19 [==============================] - 15s 834ms/step - loss: 0.6694 - accuracy: 0.6137 - val_loss: 0.7082 - val_accuracy: 0.4989
Epoch 5/1